In [1]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# time
import datetime

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

#### CKD 새로 생성
- baseline 1기, 2-7기 사이에 연달아서 2번 있으면 CKD = 1, 아니면 CKD = 0

In [6]:
final = pd.read_csv('data\\MME_.csv', encoding = 'euc-kr', low_memory = False)
final

,기수,NIHID,AGE,HEIGHT,WEIGHT,WAIST,GLU0_ORI,R_GTP_ORI,AST_ORI,ALT_ORI,...,DBP,eGFR,BMI,DRK_NEW,SMOKE_1.0,SMOKE_2.0,SMOKE_3.0,SEX_1,SEX_2,BODYFAT
0,A01,NIH1604000095,43.0,1.575,48.0,63.0,91.0,19.0,22.0,14.0,...,63.0,78.564168,19.349962,2,1,0,0,0,1,13400.0
1,A01,NIH1604000171,42.0,1.755,75.0,81.3,88.0,27.0,29.0,25.0,...,68.0,92.420166,24.350452,5,0,0,1,1,0,15700.0
2,A01,NIH1604000338,69.0,1.668,50.4,74.0,82.0,55.0,21.0,25.0,...,69.0,86.839489,18.115004,1,0,1,0,1,0,NaN
3,A01,NIH1604000362,48.0,1.556,53.6,76.4,110.0,41.0,27.0,27.0,...,74.0,102.784214,22.138368,1,1,0,0,0,1,14700.0
4,A01,NIH1604000424,47.0,1.745,88.0,94.5,81.0,8.0,17.0,14.0,...,84.0,112.380813,28.899599,4,0,0,1,1,0,19500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70205,A07,NIH1604999643,55.0,1.641,60.9,74.0,88.0,NaN,23.0,14.0,...,62.0,74.201919,22.615184,1,1,0,0,0,1,17900.0
70206,A07,NIH1604999732,75.0,1.632,50.8,81.3,110.0,NaN,55.0,15.0,...,66.0,68.312024,19.073193,3,0,1,0,1,0,10500.0
70207,A07,NIH1604999772,55.0,1.635,66.2,88.3,85.0,NaN,27.0,14.0,...,NaN,50.048887,24.764096,2,0,1,0,1,0,13900.0
70208,A07,NIH1604999929,53.0,1.530,63.2,85.3,95.0,NaN,16.0,16.0,...,71.0,74.230720,26.998163,2,1,0,0,0,1,27100.0


In [7]:
new_ckd = final[['기수', 'NIHID', 'eGFR']]
new_ckd

,기수,NIHID,eGFR
0,A01,NIH1604000095,78.564168
1,A01,NIH1604000171,92.420166
2,A01,NIH1604000338,86.839489
3,A01,NIH1604000362,102.784214
4,A01,NIH1604000424,112.380813
...,...,...,...
70205,A07,NIH1604999643,74.201919
70206,A07,NIH1604999732,68.312024
70207,A07,NIH1604999772,50.048887
70208,A07,NIH1604999929,74.230720


In [8]:
unique_id = new_ckd['NIHID'].unique().tolist()

In [9]:
# ID별로 정렬

finalDf = pd.DataFrame()
for id in unique_id: 
    df = pd.DataFrame(new_ckd.loc[new_ckd['NIHID'] == id])
    finalDf = pd.concat([finalDf, df], axis = 0)

In [10]:
finalDf.reset_index(inplace = True, drop = True)
finalDf

,기수,NIHID,eGFR
0,A01,NIH1604000095,78.564168
1,A02,NIH1604000095,NaN
2,A03,NIH1604000095,59.931748
3,A04,NIH1604000095,NaN
4,A05,NIH1604000095,56.406072
...,...,...,...
70205,A03,NIH1604999956,NaN
70206,A04,NIH1604999956,NaN
70207,A05,NIH1604999956,NaN
70208,A06,NIH1604999956,NaN


In [11]:
finalDf.loc[finalDf['eGFR'] <= 60, 'CKD'] = 1
finalDf.loc[finalDf['eGFR'] > 60, 'CKD'] = 0

In [12]:
finalDf.head(50)

,기수,NIHID,eGFR,CKD
0,A01,NIH1604000095,78.564168,0.0
1,A02,NIH1604000095,NaN,NaN
2,A03,NIH1604000095,59.931748,1.0
3,A04,NIH1604000095,NaN,NaN
4,A05,NIH1604000095,56.406072,1.0
5,A06,NIH1604000095,56.220099,1.0
6,A07,NIH1604000095,47.620724,1.0
7,A01,NIH1604000171,92.420166,0.0
8,A02,NIH1604000171,NaN,NaN
9,A03,NIH1604000171,99.390358,0.0


In [13]:
# Nan 그냥 다 빼고

finalDf.dropna(inplace = True)

In [14]:
finalDf.reset_index(inplace = True, drop = True)
finalDf.head(50)

,기수,NIHID,eGFR,CKD
0,A01,NIH1604000095,78.564168,0.0
1,A03,NIH1604000095,59.931748,1.0
2,A05,NIH1604000095,56.406072,1.0
3,A06,NIH1604000095,56.220099,1.0
4,A07,NIH1604000095,47.620724,1.0
5,A01,NIH1604000171,92.420166,0.0
6,A03,NIH1604000171,99.390358,0.0
7,A04,NIH1604000171,98.003763,0.0
8,A05,NIH1604000171,96.636512,0.0
9,A06,NIH1604000171,90.509321,0.0


In [15]:
# lst = ['A02', 'A03', 'A04', 'A05', 'A06', 'A07']

In [16]:
df = pd.DataFrame(finalDf.loc[finalDf['NIHID'] == 'NIH1604000338'].reset_index(drop = True))
df['final_CKD'] = 0
for i in range(len(df) - 1): 
    if (df['CKD'][i] == 1) & (df['CKD'][i + 1] == 1): 
        df['final_CKD'] = 1

In [17]:
df

,기수,NIHID,eGFR,CKD,final_CKD
0,A01,NIH1604000338,86.839489,0.0,1
1,A02,NIH1604000338,60.473461,0.0,1
2,A03,NIH1604000338,63.444719,0.0,1
3,A04,NIH1604000338,59.395791,1.0,1
4,A05,NIH1604000338,65.853842,0.0,1
5,A06,NIH1604000338,57.168765,1.0,1
6,A07,NIH1604000338,53.656610,1.0,1


In [18]:
ckd_df = pd.DataFrame()
for id in unique_id: 
    df = pd.DataFrame(finalDf.loc[finalDf['NIHID'] == id].reset_index(drop = True))
    df['final_CKD'] = 0
    for i in range(1, len(df) - 1): 
        if (df['CKD'][i] == 1) & (df['CKD'][i + 1] == 1): 
            df['final_CKD'] = 1
        
    ckd_df = pd.concat([ckd_df, df], axis = 0)

In [19]:
ckd_df.head()

,기수,NIHID,eGFR,CKD,final_CKD
0,A01,NIH1604000095,78.564168,0.0,1
1,A03,NIH1604000095,59.931748,1.0,1
2,A05,NIH1604000095,56.406072,1.0,1
3,A06,NIH1604000095,56.220099,1.0,1
4,A07,NIH1604000095,47.620724,1.0,1


In [20]:
k = ckd_df.copy()

In [21]:
k.drop_duplicates(inplace = True, subset = ['NIHID'])

In [22]:
k.drop(['기수', 'eGFR', 'CKD'], axis = 1, inplace = True)
k.reset_index(drop = True, inplace = True)
k

,NIHID,final_CKD
0,NIH1604000095,1
1,NIH1604000171,0
2,NIH1604000338,1
3,NIH1604000362,0
4,NIH1604000424,0
...,...,...
10025,NIH1604999643,0
10026,NIH1604999732,0
10027,NIH1604999772,1
10028,NIH1604999929,0


In [23]:
k['final_CKD'].value_counts()

0    9218
1     812
Name: final_CKD, dtype: int64

In [24]:
k.to_csv('CKD_final_Baseline1st+OccuredTwiceOrMoreInARow.csv', encoding = 'euc-kr', index = False)